## Evalatuation

In [97]:
! git clone https://github.com/boberle/corefconversion.git

Cloning into 'corefconversion'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 61 (delta 18), reused 57 (delta 14), pack-reused 0
Unpacking objects: 100% (61/61), done.


In [ ]:
!unzip out.zip

Archive:  out.zip
   creating: content/coref/data/out/
  inflating: content/coref/data/out/17696610_3.json  
  inflating: content/coref/data/out/17244351_1.json  
  inflating: content/coref/data/out/17590087_0.json  
  inflating: content/coref/data/out/17590087_2.json  
  inflating: content/coref/data/out/17608565_1.json  
  inflating: content/coref/data/out/17425782_2.json  
  inflating: content/coref/data/out/17244351_0.json  
  inflating: content/coref/data/out/17194222_1.json  
  inflating: content/coref/data/out/17447844_2.json  
  inflating: content/coref/data/out/17425782_0.json  
  inflating: content/coref/data/out/17194222_2.json  
  inflating: content/coref/data/out/17425782_4.json  
  inflating: content/coref/data/out/17696610_1.json  
  inflating: content/coref/data/out/17608565_0.json  
  inflating: content/coref/data/out/17590087_4.json  
  inflating: content/coref/data/out/17194222_3.json  
  inflating: content/coref/data/out/17244351_4.json  
  inflating: content/coref/

In [110]:
%cd corefconversion/

[Errno 2] No such file or directory: 'corefconversion/'
/content/corefconversion


In [99]:
! python jsonlines2conll.py -h

usage: jsonlines2conll [-h] [-g] [-t] [-T] -o OUTFPATH [-c CONLL_FILES]
                       [--cols COLS]
                       infpaths [infpaths ...]

convert jsonlines to conll

positional arguments:
  infpaths              input files

optional arguments:
  -h, --help            show this help message and exit
  -g, --gold            use gold clusters instead of predicted clusters
  -t, --in-tab-sep      input conll files use tab as separator
  -T, --out-tab-sep     output conll files use tab as separator
  -o OUTFPATH           output file
  -c CONLL_FILES, --conll CONLL_FILES
                        conll files to merge with, may be repeated
  --cols COLS           comma separated list of cols to include, in order
                        (default: 'sentences')


In [176]:
! python jsonlines2conll.py /content/out/17194222_0.json -o ouput.conll

In [ ]:
import json
output = json.load(open("/content/out/17194222_0.json"))

comb_text = [word for sentence in output['sentences'] for word in sentence]

def convert_mention(mention):
    start = output['subtoken_map'][mention[0]]
    end = output['subtoken_map'][mention[1]] + 1
    nmention = (start, end)
    mtext = ''.join(' '.join(comb_text[mention[0]:mention[1]+1]).split(" ##"))
    return (nmention, mtext)

seen = set()
print('Clusters:')
for cluster in output['predicted_clusters']:
    mapped = []
    for mention in cluster:
        seen.add(tuple(mention))
        mapped.append(convert_mention(mention))
    print(mapped, end=",\n")

print('\nMentions:')
for mention in output['top_spans']:
    if tuple(mention) in seen:
        continue
    print(convert_mention(mention), end=",\n")

In [187]:
import conll_transform

def merge(x):

  answer = []
  current = ""

  for item in x:
    if not item.startswith("##"):
      if current:
        answer.append(current)
        current = item
      else:
        current = item
    else:
        current = current + item[2:]

  answer.append(current)
  return answer


output = json.load(open("/content/out/17194222_0.json"))
comb_text = [word for sentence in output['sentences'] for word in sentence]

#sentences = [[] for x in range(max(output['sentence_map']) + 1)]

#for i, n in enumerate(output['sentence_map']):
#  sentences[n].append(comb_text[i]) 

#chains = output['predicted_clusters']
#mentions = [ m for chain in chains for m in chain ]

#conll_transform.textpos2sentpos(mentions, sentences)

#for sentence in sentences:
#  sent.append(merge(sentence))


docs = dict()

data = output
doc_key = data["doc_key"]
for sent in zip(*[iter(data['sentences'])]):
  print(len(sent[0]))

sents = [[list(token) for token in zip(*sent)] for sent in zip(*[iter(data['sentences'])])]
#print(sents)

chains = data['predicted_clusters']

mentions = [ m for chain in chains for m in chain ]
conll_transform.textpos2sentpos(mentions, sents)

conll_transform.write_chains(sents, chains, append=True)

docs[doc_key] = sents

#print(docs)


358
367
375
346
358
384
330
357
369
342
309
